
# Gold Notebook - Star Schema

In [0]:
# Define S3 bucket and Bronze database path
s3_bucket = "s3://r2de-bucket"
silver_db_path = "r2de_project.silver"
gold_db_path = "r2de_project.gold"


## Read Silver Data
- customer
- product
- transaction
- currency

In [0]:
silver_customer = spark.sql(f"SELECT * FROM {silver_db_path}.customer;")
silver_product = spark.sql(f"SELECT * FROM {silver_db_path}.product;")
silver_transaction = spark.sql(f"SELECT * FROM {silver_db_path}.transaction;")
silver_currency = spark.sql(f"SELECT * FROM {silver_db_path}.currency;")


## Star Schema

In [0]:
# Import Spark SQL functions module as F
from pyspark.sql import functions as F

### Create Dimension Table
- dim_customer
- dim_product

In [0]:
dim_customer = silver_customer
dim_product = silver_product

dim_customer.write.format("delta").mode("overwrite").option("path", f"{s3_bucket}/gold/dim_customer").saveAsTable(f"{gold_db_path}.dim_customer")
dim_product.write.format("delta").mode("overwrite").option("path", f"{s3_bucket}/gold/dim_product").saveAsTable(f"{gold_db_path}.dim_product")


### Create Fact Table
- fact_sales

In [0]:
df_fact = silver_transaction.join(silver_currency, on="date", how="left")

In [0]:
# 1. Add compute coloumns: sales_amount_gbp, sales_amount_thb
# sales_amount_gbp = price * quantity
# sales_amount_thb = sales_amount_gbp * gbp_thb
df_fact = (df_fact
    .withColumn("sales_amount_gbp", F.col("price") * F.col("quantity"))
    .withColumn("sales_amount_thb", F.col("sales_amount_gbp") * F.col("gbp_thb"))
)

# 2. Select fact_sales column
df_fact = (df_fact
    .select(
        F.col("transaction_id"),
        F.col("date"),
        F.col("customer_id"),
        F.col("product_id"),
        F.col("quantity"),
        F.col("price").alias("unit_price_gbp"),
        F.col("sales_amount_gbp"),
        F.col("sales_amount_thb")
    )
)

In [0]:
df_fact.write.format("delta").mode("overwrite").option("path", f"{s3_bucket}/gold/fact_sales").saveAsTable(f"{gold_db_path}.fact_sales")


## Show Sample Query

In [0]:
# This query calculates the total sales (in THB) for each country and product, 
# joining fact and dimension tables, and orders the results by total sales in descending order.

sample_query = spark.sql(f"""
    SELECT 
        c.country, 
        p.product_name, 
        ROUND(SUM(f.sales_amount_thb), 2) as total_sales
    FROM {gold_db_path}.fact_sales AS f
    JOIN {gold_db_path}.dim_customer AS c ON f.customer_id = c.customer_id
    JOIN {gold_db_path}.dim_product AS p ON f.product_id = p.product_id
    GROUP BY c.country, p.product_name
    ORDER BY total_sales DESC;
""")

display(sample_query)